<strong>Parameters:</strong>
1. Cross Validation nubmer: 5 (both inner and outer)
2. n_estimator: 10, 20, 30, ... , 190, 200
3. max_depth: 1, 2, 3, 4, 5, None
4. X <- GPS and age, female, hign.educ....

<strong>Result:</strong>
- best max depth: 3


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

In [4]:
my_data = pd.read_csv('/home/ubuntu/phenotype_and_GPS.csv', header=0)
my_data['KEY'] = my_data['KEY'].astype(str)

#### I placed NAN to mean value... should I replace it?

In [5]:
my_data.fillna(my_data.mean(), inplace=True)

In [6]:
my_data.head()

,KEY,AD,DEPRESSION_SUB,IQ,NEUROTICISM,WORRY_SUB,INSOMNIA,PTSD,SNORING,CP,...,income,married,abcd_site,vol,BMI,NIH_totcomp,NIH_flucomp,CBCL_TotProb,CBCL_Internal,CBCL_External
0,NDARINV003RTV85,-5.780160,0.002583,-2.917600,0.013433,-0.000821,-0.307129,0.000000,0.063292,-2.156561,...,8.0,1.000000,6,-1.050932,20.480539,0.632147,0.605564,-0.789382,-0.731957,-0.589077
1,NDARINV007W6H7B,9.133668,0.002060,0.369931,0.000064,-0.001469,0.438520,0.017525,0.021098,-3.683131,...,10.0,1.000000,22,1.293196,18.234286,1.069812,0.887103,0.268149,0.534371,-0.418541
2,NDARINV00BD7VDC,2.059180,0.006631,-4.261953,0.012193,-0.016278,0.071778,0.010899,0.037388,-4.160142,...,10.0,1.000000,7,1.129021,16.329800,0.632147,1.544026,-0.065808,-0.008341,-0.248005
3,NDARINV00J52GPG,8.591499,0.004259,-2.582771,0.013623,0.000487,-0.328521,0.019801,0.022991,-4.682147,...,6.0,1.522359,17,0.020277,19.061896,0.329044,0.237332,-0.399765,-0.912861,-0.589077
4,NDARINV00NPMHND,3.104598,0.000878,-3.145087,0.013695,-0.013129,0.014641,0.000000,-0.008245,-4.843486,...,8.0,1.000000,17,0.663349,17.663472,-0.680849,-1.365205,1.325680,3.247931,-0.248005


#### CONSTANTS:

In [7]:
k_fold = 5

num_samples = len(my_data)
each_fold_size = int(num_samples/k_fold) +1

print('num_samples: ', num_samples)
print('fold_size: ', each_fold_size)

num_samples:  4567
fold_size:  914


#### PARAMETERS:

In [9]:
X = my_data.iloc[:, 1:35]
X.head()

,AD,DEPRESSION_SUB,IQ,NEUROTICISM,WORRY_SUB,INSOMNIA,PTSD,SNORING,CP,excl23andMe,...,HAPPINESS,age,female,race,high.educ,income,married,abcd_site,vol,BMI
0,-5.780160,0.002583,-2.917600,0.013433,-0.000821,-0.307129,0.000000,0.063292,-2.156561,-0.248034,...,0.002318,131,1,1.0,13.0,8.0,1.000000,6,-1.050932,20.480539
1,9.133668,0.002060,0.369931,0.000064,-0.001469,0.438520,0.017525,0.021098,-3.683131,-1.858655,...,-0.000128,126,0,1.0,19.0,10.0,1.000000,22,1.293196,18.234286
2,2.059180,0.006631,-4.261953,0.012193,-0.016278,0.071778,0.010899,0.037388,-4.160142,-2.382114,...,-0.001354,112,0,1.0,20.0,10.0,1.000000,7,1.129021,16.329800
3,8.591499,0.004259,-2.582771,0.013623,0.000487,-0.328521,0.019801,0.022991,-4.682147,-1.212850,...,0.001577,110,0,5.0,21.0,6.0,1.522359,17,0.020277,19.061896
4,3.104598,0.000878,-3.145087,0.013695,-0.013129,0.014641,0.000000,-0.008245,-4.843486,-1.088557,...,-0.000256,118,1,1.0,16.0,8.0,1.000000,17,0.663349,17.663472


In [10]:
y = my_data['NIH_totcomp']
y.head()

0    0.632147
1    1.069812
2    0.632147
3    0.329044
4   -0.680849
Name: NIH_totcomp, dtype: float64

In [11]:
X = X.to_numpy()
y = y.to_numpy().ravel()

In [12]:
inner_cv = KFold(n_splits = k_fold, shuffle=False, random_state = 123)
outer_cv = KFold(n_splits = k_fold, shuffle=False, random_state = 123)

In [13]:
outerloop = 0

for train_index, test_index in outer_cv.split(X, y):
    outerloop += 1
    print('----------------------------')
    print('Loop: ', outerloop)
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    params = {'n_estimators': np.arange(10, 200, 10),
              'max_depth': [1, 2, 3, 4, 5, None]}
    
    rf = RandomForestRegressor()
    rf_grid = GridSearchCV(estimator = rf, param_grid = params, cv = inner_cv, scoring='r2', n_jobs=-1)
    
    rf_grid.fit(X_train, y_train)
    y_pred_test = rf_grid.predict(X_test)
    #y_pred_train = rf_grid.predict(X_train)
    
    
    print('best train parameter: ', rf_grid.best_params_)
    print('best train score: ', rf_grid.best_score_)
    print('\n')
    print('test score: ', r2_score(y_test, y_pred_test))
    print('----------------------------')

----------------------------
Loop:  1
best train parameter:  {'max_depth': 5, 'n_estimators': 180}
best train score:  0.1936660857573337


test score:  0.13303135858821624
----------------------------
----------------------------
Loop:  2
best train parameter:  {'max_depth': 5, 'n_estimators': 190}
best train score:  0.1798805034957367


test score:  0.19129920307446846
----------------------------
----------------------------
Loop:  3
best train parameter:  {'max_depth': 5, 'n_estimators': 70}
best train score:  0.18522502264234528


test score:  0.1650408684453636
----------------------------
----------------------------
Loop:  4
best train parameter:  {'max_depth': 5, 'n_estimators': 120}
best train score:  0.16949991572992712


test score:  0.22497094003563056
----------------------------
----------------------------
Loop:  5
best train parameter:  {'max_depth': 5, 'n_estimators': 90}
best train score:  0.18011338822661743


test score:  0.1985734005356733
-------------------------

##### score is R2 score